In [20]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import math
import random
from geopy.distance import geodesic 
from geopy.distance import distance


#Calculates a penalty
def get_penalty(weight, max_weight, penalty_factor):
    if weight <= max_weight:
        return 0
    else:
        excess_weight = weight - max_weight
        penalty = penalty_factor * excess_weight
        return penalty

#Calculates the probabilities of selecting the next point
def get_probabilities(pheromone, points, current_point, visited, weights, capacity, alpha, beta, penalty_factor):
    unvisited = np.where(np.logical_not(visited))[0]
    probabilities = np.zeros(len(unvisited))
    total_weight = np.sum(weights[unvisited])
    for i, unvisited_point in enumerate(unvisited):
        if weights[unvisited_point] + total_weight <= capacity:
            probabilities[i] = (pheromone[current_point, unvisited_point] ** alpha) / (distance(points[current_point], points[unvisited_point]) ** beta)
        else:
            probabilities[i] = (pheromone[current_point, unvisited_point] ** alpha) / (penalty_factor * distance(points[current_point], points[unvisited_point]) ** beta)
    return probabilities / np.sum(probabilities)

#Calculates the Euclidean distance between two points
def distance(point1, point2):
    return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)
    
#Updates pheromones based on found paths
def update_pheromone(pheromone, paths, path_lengths, Q):
    for path, path_length in zip(paths, path_lengths):
        for i in range(len(path)-1):
            pheromone[path[i], path[i+1]] += Q / path_length
            pheromone[path[-1], path[0]] += Q / path_length
    return pheromone
    


In [40]:

def ant_algorithm(points, num, weight, capacity):

    iterations = 100
    ants = 10
    alpha = 1
    beta = 1
    evaporation_rate = 0.5
    Q = 1 
    n_points = len(points)
    penalty_factor = 1000

    start_point = points[0]
    index_start_point = 0
    n_points = len(points)
    pheromone = np.ones((n_points, n_points))
    best_path = [None] * num
    best_weight = [None] * num 
    weights = waga

    best_path_length = np.inf
    
    for iteration in range(iterations):
        paths = [[]] * num 
        path_lengths = [[]]*num

        for ant in range(ants):
            visited = [False]*n_points
            current_point = index_start_point
            visited[current_point] = True
            
            path = [[current_point]] * num
            path_length = [0]*num
            weight=[0]*num

            while False in visited:
                unvisited = np.where(np.logical_not(visited))[0]
                probabilities = get_probabilities(pheromone, points, current_point, visited, weights, capacity, alpha, beta, penalty_factor)


                next_point = np.random.choice(unvisited, p=probabilities)

                
                for i in range(num):
                    if (weight[i] + waga[next_point] <capacity and len(path[i]) < (n_points-1)/num+1):
                        path[i] = path[i] + [next_point]
                        path_length[i] += distance(points[current_point], points[next_point])
                        weight[i] += waga[next_point]
                        break 
                        
                    
                visited[next_point] = True
                current_point = next_point
            
            for i in range(num):
                paths[i] = paths[i]+[path[i]]
                path_lengths[i]=path_lengths[i]+[path_length[i]]
            

        total_path_length = sum([sum(sublist) for sublist in path_lengths])

        pathh = []
        weightt = []
        
        for i in range(num):
            pathh.append(path[i])
            weightt.append(weight[i])
        

        if total_path_length < best_path_length:
            for i in range(len(pathh)):
                best_path[i] = pathh[i]
                best_weight[i] = weightt[i]
                
        best_path_length = total_path_length
            
        pheromone *= evaporation_rate
        for i in range(num):
            pheromone = update_pheromone(pheromone, paths[i], path_lengths[i], Q)
            
    
    result_rows = []

    for i in range(len(best_path)):
        path = best_path[i]
        path_num = i + 1  

        path_coords = [points[point] for point in path]
        path_coords.append(start_point)


        path_tuples = [tuple(coords) for coords in path_coords]
        
        path_distances = [distance(path_coords[j], path_coords[j+1]) for j in range(len(path_coords)-1)]

        total_distance = sum(path_distances)

        result_rows.append({'Driver': path_num, 'Queue': path_tuples, 'Distance': total_distance})

    #Create a dataframe from a list of rows
    result_df = pd.DataFrame(result_rows, columns=['Driver', 'Queue', 'Distance'])

    return result_df


In [41]:
r_choice = [1]*12+[2]*7+[3]*3
weights = np.array(r_choice)
weights = np.insert(waga, 0,0)

In [42]:
points = tuple((random.uniform(1.0, 20.0), random.uniform(1.0, 20.0)) for _ in range(22))
result = ant_algorithm(points, 2, weights, 10)

In [43]:
result

,Driver,Queue,Distance
0,1,"[(6.525249084953096, 12.835631288328857), (6.9...",25.217109
1,2,"[(6.525249084953096, 12.835631288328857), (9.6...",33.999598
